## Importación de Librerías

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from connDef import *

### Obtención de datos e inserción a dataframes

In [2]:
# Conexión a la base de datos.

conn = dbconn()

db connected



Read SQL -> df = pd.read_sql(con <connection>, sql = db.text(<query>) )


In [3]:
query = db.text('SELECT * FROM DcDataLakeDB.CoffeShopData')
coffeData = pd.read_sql(con = conn.connect(), sql = query)

In [4]:
query = db.text('SELECT * FROM DcDataLakeDB.Customer')
Customer = pd.read_sql(con = conn.connect(), sql = query)

In [5]:
query = db.text('SELECT * FROM DcDataLakeDB.Dates')
dates = pd.read_sql(con = conn.connect(), sql = query)

In [6]:
query = db.text('SELECT * FROM DcDataLakeDB.Generations')
generations = pd.read_sql(con = conn.connect(), sql = query)

In [7]:
query = db.text('SELECT * FROM DcDataLakeDB.Pastry_inventory')
pastryInventory = pd.read_sql(con = conn.connect(), sql = query)

In [8]:
query = db.text('SELECT * FROM DcDataLakeDB.Product')
product = pd.read_sql(con = conn.connect(), sql = query)

In [9]:
query = db.text('SELECT * FROM DcDataLakeDB.Sales_target')
salesTarget = pd.read_sql(con = conn.connect(), sql = query)

In [10]:
query = db.text('SELECT * FROM DcDataLakeDB.SalesOutlet')
salesOutlet = pd.read_sql(con = conn.connect(), sql = query)

In [11]:
query = db.text('SELECT * FROM DcDataLakeDB.Staff')
staff = pd.read_sql(con = conn.connect(), sql = query)

## SuperData Dataframe

JOIN DATAFRAMES:
- coffedata
- customer
- dates 
- product
- sales outlet

In [12]:
coffeData.head(3)

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,orders,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item
0,7,01/04/2019,12:04:43,3,12,558,N,1,1,52,1,2.5,2.5,N
1,11,01/04/2019,15:54:39,3,17,781,N,1,1,27,2,7.0,3.5,N
2,19,01/04/2019,14:34:59,3,17,788,Y,1,1,46,2,5.0,2.5,N


In [13]:
superData = coffeData.join(dates.set_index('transaction_date'), on='transaction_date')

In [14]:
superData = superData.join(product.set_index('product_id') ,on='product_id')

In [15]:
superData = superData.join(salesOutlet.set_index('sales_outlet_id'), on='sales_outlet_id')

In [16]:
superData = superData.join(Customer.set_index('customer_id'), on='customer_id')

In [17]:
superData.shape

(49894, 52)

In [18]:
superData[['current_retail_price', 'unit_price']].head(2)

,current_retail_price,unit_price
0,$2.50,2.5
1,$3.50,3.5


In [19]:
superData = superData.drop(columns = ['promo_yn', 'unit_price'])

In [20]:
superData.columns

Index(['transaction_id', 'transaction_date', 'transaction_time',
       'sales_outlet_id', 'staff_id', 'customer_id', 'instore_yn', 'orders',
       'line_item_id', 'product_id', 'quantity', 'line_item_amount',
       'promo_item', 'date_id', 'week_id', 'week_desc', 'month_id',
       'month_name', 'quarter_id', 'quarter_name', 'year_id', 'product_group',
       'product_category', 'product_type', 'product', 'product_description',
       'unit_of_measure', 'current_wholesale_price', 'current_retail_price',
       'tax_exempt_yn', 'new_product_yn', 'sales_outlet_type',
       'store_square_feet', 'store_address', 'store_city',
       'store_state_province', 'store_telephone', 'store_postal_code',
       'store_longitude', 'store_latitude', 'manager', 'neighborhood',
       'home_store', 'customer_first_name', 'customer_email', 'customer_since',
       'loyalty_card_number', 'birthdate', 'gender', 'birth_year'],
      dtype='object')

## Engineering

### Feature Engineering:

- 1.- Customers by day by store
- 2.- Volume by day by store
- 3.- Average price per item by store
- 4.- Loyalty binary 0 1
- 5.- Average price by product group
- 6.- Total volume by customer by store
- 7.- Total revenue by customer by store
- 8.- Before midday, aftermiday binary columns 0 1
- 9.- Proportion of sales in store by store
- 10.- Number of orders by store by day
- 11.- Number of products sold by store
- 12.- Proportion of items in promo by store
- 13.- Propotion of customer with loyalty by store
- 14.- Transaction in promotion binary 0 1

In [21]:
#Customers by day by store

In [34]:
superData['loyalty_card_number'].isnull().sum()

25042

In [41]:
#Loyalty binary 0 1
df = superData[['transaction_id', 'loyalty_card_number']]
df['Loyalty'] = df['loyalty_card_number'].notnull().astype(int)
df.drop(['loyalty_card_number'], axis=1, inplace=True)
df.head(10)

/tmp/ipykernel_259368/669938501.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Loyalty'] = df['loyalty_card_number'].notnull().astype(int)
/tmp/ipykernel_259368/669938501.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['loyalty_card_number'], axis=1, inplace=True)


,transaction_id,Loyalty
0,7,1
1,11,1
2,19,1
3,32,1
4,33,1
5,39,1
6,50,1
7,53,1
8,59,1
9,62,1
